In [64]:
import pandas as pd
import configparser
import os

# Read the value from the config.ini file
config = configparser.ConfigParser()
try:
    config.read('config.ini')
    print(config.get('data', 'file_path1'))
except:
    print('Error reading the config file')

try:
    file_path = config.get('data', 'file_path1')
    if not os.path.isfile(file_path):
        raise FileNotFoundError('File not found')
    # Read the data from the csv file
    df = pd.read_csv(file_path)

except FileNotFoundError as e:
    file_path = config.get('data', 'file_path2')
    if not os.path.isfile(file_path):
        raise FileNotFoundError('File not found')
    # Read the data from the csv file
    df = pd.read_csv(file_path)
except:
    print('Error reading the file path from the config file')

if df is not None:
    print(df.head())
else:
    print('DataFrame is not defined')

C:\\DataExport\\VN30F1M.csv
    Symbol    Date   Open   High    Low  Close  Volume           RSI  \
0  VN30F1M  170810  757.9  757.9  745.5  745.9     367 -1.000000e+10   
1  VN30F1M  170811  746.0  746.4  743.4  745.7     510 -1.000000e+10   
2  VN30F1M  170814  745.9  748.9  745.7  748.9     707 -1.000000e+10   
3  VN30F1M  170815  750.3  750.9  748.0  748.0     946 -1.000000e+10   
4  VN30F1M  170816  747.7  748.4  745.1  748.1    1065 -1.000000e+10   

           MACD        Signal          MA10          MA20         MA100  \
0 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   
1 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   
2 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   
3 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   
4 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   

          MA200           MFI      Ultimate  
0 -1.000000e+10 -1.000000e+10 -1.000000e+1

In [81]:
# Drop the rows with missing values
df.dropna(inplace=True)
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
Index: 1768 entries, 1 to 1768
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Symbol      1768 non-null   object        
 1   Date        1768 non-null   datetime64[ns]
 2   Open        1768 non-null   float64       
 3   High        1768 non-null   float64       
 4   Low         1768 non-null   float64       
 5   Close       1768 non-null   float64       
 6   Volume      1768 non-null   int64         
 7   RSI         1768 non-null   float64       
 8   MACD        1768 non-null   float64       
 9   Signal      1768 non-null   float64       
 10  MA10        1768 non-null   float64       
 11  MA20        1768 non-null   float64       
 12  MA100       1768 non-null   float64       
 13  MA200       1768 non-null   float64       
 14  MFI         1768 non-null   float64       
 15  Ultimate    1768 non-null   float64       
 16  DoW         1768 non-null   o

,Symbol,Date,Open,High,Low,Close,Volume,RSI,MACD,Signal,MA10,MA20,MA100,MA200,MFI,Ultimate,DoW,Prev_Close,Diff,Action
1,VN30F1M,2017-08-11,746.0,746.4,743.4,745.7,510,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Fr,745.9,-2.5,1
2,VN30F1M,2017-08-14,745.9,748.9,745.7,748.9,707,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Mo,745.7,0.0,0
3,VN30F1M,2017-08-15,750.3,750.9,748.0,748.0,946,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Tu,748.9,-0.9,1
4,VN30F1M,2017-08-16,747.7,748.4,745.1,748.1,1065,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,We,748.0,-2.9,0
5,VN30F1M,2017-08-17,748.5,749.8,744.5,744.6,1134,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Th-exp,748.1,-3.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764,VN30F1M,2024-08-29,1321.1,1330.2,1320.3,1326.8,165014,6.732316e+01,1.081585e+01,8.429009e+00,1.311760e+03,1.285190e+03,1.284258e+03,1.232720e+03,9.568903e+01,6.558941e+01,Th,1322.1,-1.8,1
1765,VN30F1M,2024-08-30,1328.9,1335.0,1328.0,1332.7,123214,6.936446e+01,1.204346e+01,1.018499e+01,1.316230e+03,1.288030e+03,1.285043e+03,1.233842e+03,9.552423e+01,6.217440e+01,Fr,1326.8,1.2,0
1766,VN30F1M,2024-09-04,1320.8,1321.6,1307.3,1314.0,174577,5.717341e+01,1.137627e+01,1.127575e+01,1.317830e+03,1.292080e+03,1.285541e+03,1.234876e+03,8.829455e+01,5.352198e+01,We,1332.7,-25.4,0
1767,VN30F1M,2024-09-05,1316.1,1318.6,1304.2,1307.7,168132,5.374620e+01,1.022134e+01,1.171073e+01,1.318400e+03,1.294985e+03,1.286017e+03,1.235815e+03,8.134280e+01,4.992623e+01,Th,1314.0,-9.8,1


In [86]:
import pandas as pd

# Select the relevant columns
columns_of_interest = ['RSI', 'MACD', 'Signal', 'MA10', 'MA20', 'MA200', 'MFI', 'Ultimate', 'Close']
df_selected = df[columns_of_interest]
# Clear all rows which have -1.000000e+10 in any column
df_selected = df_selected[~df_selected.isin([-1.000000e+10]).any(axis=1)]

# Calculate the correlation matrix
correlation_matrix = df_selected.corr()

# Extract the correlation values for the 'Close' column
correlation_with_close = correlation_matrix['Close'].drop('Close')

# Print the correlation values
print(correlation_with_close)

RSI         0.158700
MACD        0.225044
Signal      0.232463
MA10        0.993778
MA20        0.986614
MA200       0.787834
MFI         0.145721
Ultimate    0.108933
Name: Close, dtype: float64


In [66]:
import pandas as pd
import numpy as np

# Assuming your dataframe is named "df"
# Convert the "Date" column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Map the day of the week to corresponding labels
day_labels = ['Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa', 'Su']
df['DoW'] = df['Date'].dt.dayofweek.map(lambda x: day_labels[x])

# Check if the date is the 3rd Thursday of the month and label it as "Th-expired"
DoWExp = config.get('derivatives', 'DoWExp')

df['DoW'] = np.where((df['Date'].dt.weekday == 3) & (df['Date'].dt.day <= 21) & (df['Date'].dt.day >= 15), DoWExp + '-exp', df['DoW'])

# Revert the "Date" column back to the original format
df['Date'] = df['Date'].dt.strftime('%y%m%d')

# Print the updated dataframe
df.head()


,Symbol,Date,Open,High,Low,Close,Volume,RSI,MACD,Signal,MA10,MA20,MA100,MA200,MFI,Ultimate,DoW
0,VN30F1M,170810,757.9,757.9,745.5,745.9,367,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Th
1,VN30F1M,170811,746.0,746.4,743.4,745.7,510,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Fr
2,VN30F1M,170814,745.9,748.9,745.7,748.9,707,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Mo
3,VN30F1M,170815,750.3,750.9,748.0,748.0,946,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Tu
4,VN30F1M,170816,747.7,748.4,745.1,748.1,1065,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,We


In [67]:
import pandas as pd

# Method 1: Buying at the close and selling at the open the next day
df_method1 = df.copy()
df_method1['Entry'] = df_method1['Close'].shift()
df_method1['Exit'] = df_method1['Open']
df_method1['Points'] = df_method1['Exit'] - df_method1['Entry']
cumulative_points_method1 = df_method1['Points'].cumsum().iloc[-1]

# Method 2: Shorting at the open plus 2 points and covering at the close
df_method2 = df.copy()
df_method2['Entry'] = df_method2['Open'] + 2
df_method2['Exit'] = df_method2['Close']
df_method2['Points'] = df_method2['Exit'] - df_method2['Entry']
cumulative_points_method2 = df_method2['Points'].cumsum().iloc[-1]

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']
cumulative_points_method3 = df_method3['Points'].cumsum().iloc[-1]

# Method 4: Buying at open and selling at close if previous day have Close > MA(close, 10). Shorting at open and covering at close if previous day have Close < MA(close, 10)
df_method4 = df.copy()
df_method4['MA10'] = df_method4['Close'].rolling(window=10).mean()
df_method4['Entry'] = df_method4['Open']
df_method4['Exit'] = df_method4['Close']
df_method4['Points'] = np.where(df_method4['Close'].shift() > df_method4['MA10'].shift(), df_method4['Exit'] - df_method4['Entry'], df_method4['Entry'] - df_method4['Exit'])
cumulative_points_method4 = df_method4['Points'].cumsum().iloc[-1]

# Method 5: Buying at the previous close minus 2 points and selling at the close
df_method5 = df.copy()
df_method5['Entry'] = df_method5['Close'].shift() - 2
df_method5['Exit'] = df_method5['Close']
df_method5['Points'] = df_method5['Exit'] - df_method5['Entry']
cumulative_points_method5 = df_method5['Points'].cumsum().iloc[-1]

# Compare the cumulative points for each method
method_points = {
    'Method 1': cumulative_points_method1,
    'Method 2': cumulative_points_method2,
    'Method 3': cumulative_points_method3,
    'Method 4': cumulative_points_method4,
    'Method 5': cumulative_points_method5
}

# Find the method with the most points
best_method = max(method_points, key=method_points.get)
best_points = method_points[best_method]

# Print the results
for method, points in method_points.items():
    print(f"{method}: Cumulative Points = {points}")

print(f"\nBest Method: {best_method}")
print(f"Cumulative Points: {best_points}")


Method 1: Cumulative Points = 778.6999999999994
Method 2: Cumulative Points = -3765.6000000000017
Method 3: Cumulative Points = 3310.3999999999974
Method 4: Cumulative Points = 503.1999999999972
Method 5: Cumulative Points = 4099.099999999999

Best Method: Method 5
Cumulative Points: 4099.099999999999


In [68]:
# Optimize the moving average window length in Method 4
cumulative_points = {}
for window in range(5, 21):
    df_method4 = df.copy()
    df_method4['MA'] = df_method4['Close'].rolling(window=window).mean()
    df_method4['Entry'] = df_method4['Open']
    df_method4['Exit'] = df_method4['Close']
    df_method4['Points'] = np.where(df_method4['Close'].shift() > df_method4['MA'].shift(), df_method4['Exit'] - df_method4['Entry'], df_method4['Entry'] - df_method4['Exit'])
    cumulative_points[window] = df_method4['Points'].cumsum().iloc[-1]

# Find the Range length with the most points
best_window = max(cumulative_points, key=cumulative_points.get)
best_points = cumulative_points[best_window]

# Print the results
for window, points in cumulative_points.items():
    print(f"Range Length: {window}, Cumulative Points = {points}")


Range Length: 5, Cumulative Points = 686.9999999999999
Range Length: 6, Cumulative Points = 908.5999999999991
Range Length: 7, Cumulative Points = 934.1999999999985
Range Length: 8, Cumulative Points = 1039.3999999999985
Range Length: 9, Cumulative Points = 663.7999999999973
Range Length: 10, Cumulative Points = 503.1999999999972
Range Length: 11, Cumulative Points = 600.1999999999979
Range Length: 12, Cumulative Points = 415.1999999999979
Range Length: 13, Cumulative Points = 346.599999999998
Range Length: 14, Cumulative Points = 229.79999999999734
Range Length: 15, Cumulative Points = 204.79999999999666
Range Length: 16, Cumulative Points = 183.59999999999707
Range Length: 17, Cumulative Points = 276.39999999999634
Range Length: 18, Cumulative Points = 293.5999999999964
Range Length: 19, Cumulative Points = 358.1999999999956
Range Length: 20, Cumulative Points = 429.79999999999484


In [69]:
import pandas as pd

# Convert the Date column to string format
df['Date'] = df['Date'].astype(str)

# Filter the dataframe for the desired date range (2022 and 2024)
df_filtered = df[(df['Date'] >= '220101') & (df['Date'] <= '240906')]

# Method 1: Buying at the close and selling at the open the next day
df_method1 = df_filtered.copy()
df_method1['Entry'] = df_method1['Close'].shift()
df_method1['Exit'] = df_method1['Open']
df_method1['Points'] = df_method1['Exit'] - df_method1['Entry']

# Calculate cumulative points for Method 1
if len(df_method1) > 0:
    cumulative_points_method1 = df_method1['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method1 = 0

# Method 2: Shorting at the open plus 2 points and covering at the close
df_method2 = df_filtered.copy()
df_method2['Entry'] = df_method2['Open'] + 2
df_method2['Exit'] = df_method2['Close']
df_method2['Points'] = df_method2['Exit'] - df_method2['Entry']

# Calculate cumulative points for Method 2
if len(df_method2) > 0:
    cumulative_points_method2 = df_method2['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method2 = 0

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df_filtered.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']

# Calculate cumulative points for Method 3
if len(df_method3) > 0:
    cumulative_points_method3 = df_method3['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method3 = 0

# Method 4: Buying at open and selling at close if previous day have Close > MA(close, 5). Shorting at open and covering at close if previous day have Close < MA(close, 5)
df_method4 = df_filtered.copy()
df_method4['MA5'] = df_method4['Close'].rolling(window=5).mean()
df_method4['Entry'] = df_method4['Open']
df_method4['Exit'] = df_method4['Close']
df_method4['Points'] = np.where(df_method4['Close'].shift() > df_method4['MA5'].shift(), df_method4['Exit'] - df_method4['Entry'], df_method4['Entry'] - df_method4['Exit'])

# Calculate cumulative points for Method 4
if len(df_method4) > 0:
    cumulative_points_method4 = df_method4['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method4 = 0

# Method 5: Buying at the previous close minus 2 points and selling at the close
df_method5 = df_filtered.copy()
df_method5['Entry'] = df_method5['Close'].shift() - 2
df_method5['Exit'] = df_method5['Close']
df_method5['Points'] = df_method5['Exit'] - df_method5['Entry']

# Calculate cumulative points for Method 5
if len(df_method5) > 0:
    cumulative_points_method5 = df_method5['Points'].cumsum().iloc[-1]

# Compare the cumulative points for each method
method_points = {
    'Method 1': cumulative_points_method1,
    'Method 2': cumulative_points_method2,
    'Method 3': cumulative_points_method3,
    'Method 4': cumulative_points_method4,
    'Method 5': cumulative_points_method5
}

# Find the method with the most points
best_method = max(method_points, key=method_points.get)
best_points = method_points[best_method]

# Print the results
for method, points in method_points.items():
    print(f"{method}: Cumulative Points = {points}")

print(f"\nBest Method: {best_method}")
print(f"Cumulative Points: {best_points}")


Method 1: Cumulative Points = 236.40000000000214
Method 2: Cumulative Points = -1804.300000000002
Method 3: Cumulative Points = 859.6999999999978
Method 4: Cumulative Points = 294.9000000000001
Method 5: Cumulative Points = 1080.5

Best Method: Method 5
Cumulative Points: 1080.5


In [70]:
### METHOD 3
# Method 3: Buying at the open minus 2 points and selling at the close
import pandas as pd

# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Filter the dataframe for the desired date range (2022 and 2023 and 2024)
df_filtered = df[(df['Date'].dt.year.isin([2022, 2023, 2024]))]

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df_filtered.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']

# Calculate cumulative points for Method 3
cumulative_points_method3 = df_method3['Points'].cumsum()

# Group the data by month and year and calculate the points earned for each month and year
df_method3['Month'] = df_method3['Date'].dt.month
df_method3['Year'] = df_method3['Date'].dt.year
monthly_points = df_method3.groupby(['Year', 'Month'])['Points'].sum()

# Print the points earned each month and year
#print("Points Earned Each Month and Year:")
#print(monthly_points)


# Show results in year 2024 only
monthly_points_2024 = monthly_points[2024]
print("Points Earned Each Month in 2024:")
print(monthly_points_2024)

Points Earned Each Month in 2024:
Month
1     62.8
2    101.9
3     55.2
4      0.3
5     60.3
6     13.0
7     56.2
8     66.9
9     -7.8
Name: Points, dtype: float64


In [71]:
### METHOD 4
# Method 4: Buying at open and selling at close if previous day have Close > MA(close, 5). Shorting at open and covering at close if previous day have Close < MA(close, 5)
import pandas as pd

# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Filter the dataframe for the desired date range (2022 and 2023 and 2024)
df_filtered = df[(df['Date'].dt.year.isin([2022, 2023, 2024]))]

# Method 4: Buying at open and selling at close if previous day have Close > MA(close, 5). Shorting at open and covering at close if previous day have Close < MA(close, 5)
df_method4 = df_filtered.copy()
df_method4['MA5'] = df_method4['Close'].rolling(window=5).mean()
df_method4['Entry'] = df_method4['Open']
df_method4['Exit'] = df_method4['Close']
df_method4['Points'] = np.where(df_method4['Close'].shift() > df_method4['MA5'].shift(), df_method4['Exit'] - df_method4['Entry'], df_method4['Entry'] - df_method4['Exit'])

# Calculate cumulative points for Method 4
cumulative_points_method4 = df_method4['Points'].cumsum()

# Group the data by month and year and calculate the points earned for each month and year
df_method4['Month'] = df_method4['Date'].dt.month
df_method4['Year'] = df_method4['Date'].dt.year
monthly_points = df_method4.groupby(['Year', 'Month'])['Points'].sum()

# Print the points earned each month and year
#print("Points Earned Each Month and Year:")
#print(monthly_points)

# Show results in year 2024 only
monthly_points_2024 = monthly_points[2024]
print("Points Earned Each Month in 2024:")
print(monthly_points_2024)

Points Earned Each Month in 2024:
Month
1   -25.8
2    58.7
3   -73.2
4    22.7
5    10.5
6   -13.4
7    39.2
8    29.1
9     0.2
Name: Points, dtype: float64


In [80]:
### METHOD 5
# Method 5: Buying at the previous close minus 2 points and selling at the close

import pandas as pd

# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Filter the dataframe for the desired date range (2022 and 2023 and 2024)
df_filtered = df[(df['Date'].dt.year.isin([2022, 2023, 2024]))]

# Method 5: Buying at the previous close minus 2 points and selling at the close
df_method5 = df_filtered.copy()
df_method5['Entry'] = df_method5['Close'].shift() - 2
df_method5['Exit'] = df_method5['Close']
df_method5['Points'] = df_method5['Exit'] - df_method5['Entry']

# Calculate cumulative points for Method 5
cumulative_points_method5 = df_method5['Points'].cumsum()

# Group the data by month and year and calculate the points earned for each month and year
df_method5['Month'] = df_method5['Date'].dt.month
df_method5['Year'] = df_method5['Date'].dt.year
monthly_points = df_method5.groupby(['Year', 'Month'])['Points'].sum()

# Print the points earned each month and year
#print("Points Earned Each Month and Year:")
#print(monthly_points)

# Show results in year 2024 only
monthly_points_2024 = monthly_points[2024]
print("Points Earned Each Month in 2024:")
print(monthly_points_2024)

Points Earned Each Month in 2024:
Month
1     80.4
2    123.8
3     77.3
4    -27.7
5     81.6
6     44.3
7     76.0
8     72.4
9    -17.7
Name: Points, dtype: float64


In [73]:
# Create a new column 'Prev_Close' which is the 'Close' column shifted by 1
df['Prev_Close'] = df['Close'].shift(1)

# Calculate the difference between 'Low' and 'Prev_Close'
df['Diff'] = df['Low'] - df['Prev_Close']

# Filter data for the year 2023
df_2023 = df[df['Date'].dt.year == 2023]

# Calculate the rate at which 'Low' is less than 'Prev_Close' by 2 points
rate = (df_2023['Diff'] <= -2).mean()

print(f"The rate at which 'Low' is less than the previous day's 'Close' by 2 points in the trading days of 2023 is {rate}")

The rate at which 'Low' is less than the previous day's 'Close' by 2 points in the trading days of 2023 is 0.714859437751004


In [74]:
# Create a new column 'Prev_Close' which is the 'Close' column shifted by 1
df['Prev_Close'] = df['Close'].shift(1)

# Calculate the difference between 'Low' and 'Prev_Close'
df['Diff'] = df['Low'] - df['Prev_Close']

# Filter data for the year 2023
df_2023 = df[df['Date'].dt.year == 2023]

# Calculate the rate at which 'Low' is less than 'Prev_Close' by 2 to 10 points
for i in range(2, 11):
    rate = (df_2023['Diff'] <= -i).mean()
    print(f"The rate at which 'Low' is less than the previous day's 'Close' by {i} points in the trading days of 2023 is {rate}")

The rate at which 'Low' is less than the previous day's 'Close' by 2 points in the trading days of 2023 is 0.714859437751004
The rate at which 'Low' is less than the previous day's 'Close' by 3 points in the trading days of 2023 is 0.6345381526104418
The rate at which 'Low' is less than the previous day's 'Close' by 4 points in the trading days of 2023 is 0.5502008032128514
The rate at which 'Low' is less than the previous day's 'Close' by 5 points in the trading days of 2023 is 0.4939759036144578
The rate at which 'Low' is less than the previous day's 'Close' by 6 points in the trading days of 2023 is 0.43373493975903615
The rate at which 'Low' is less than the previous day's 'Close' by 7 points in the trading days of 2023 is 0.40160642570281124
The rate at which 'Low' is less than the previous day's 'Close' by 8 points in the trading days of 2023 is 0.3614457831325301
The rate at which 'Low' is less than the previous day's 'Close' by 9 points in the trading days of 2023 is 0.33734939

In [75]:
# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Filter data for the year 2024
df_2024 = df[df['Date'].dt.year == 2024]

# Calculate the rate at which 'Low' is less than 'Prev_Close' by 2 to 10 points
for i in range(2, 11):
    rate = (df_2024['Diff'] <= -i).mean()
    print(f"The rate at which 'Low' is less than the previous day's 'Close' by {i} points in the trading days of 2024 is {rate}")

The rate at which 'Low' is less than the previous day's 'Close' by 2 points in the trading days of 2024 is 0.7023809523809523
The rate at which 'Low' is less than the previous day's 'Close' by 3 points in the trading days of 2024 is 0.625
The rate at which 'Low' is less than the previous day's 'Close' by 4 points in the trading days of 2024 is 0.5357142857142857
The rate at which 'Low' is less than the previous day's 'Close' by 5 points in the trading days of 2024 is 0.48214285714285715
The rate at which 'Low' is less than the previous day's 'Close' by 6 points in the trading days of 2024 is 0.4166666666666667
The rate at which 'Low' is less than the previous day's 'Close' by 7 points in the trading days of 2024 is 0.38095238095238093
The rate at which 'Low' is less than the previous day's 'Close' by 8 points in the trading days of 2024 is 0.35119047619047616
The rate at which 'Low' is less than the previous day's 'Close' by 9 points in the trading days of 2024 is 0.3333333333333333
Th

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Create a target variable 'Action' that is 1 if the price increased compared to the previous day, and 0 otherwise
df['Action'] = (df['Close'].shift(-1) > df['Close']).astype(int)

# Use the 'Open', 'High', 'Low', 'Close' columns as features
features = df[['Open', 'High', 'Low', 'Close']]

# Use the 'Action' column as the target
target = df['Action']

# Split the data into a training set and a test set
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

# Create and train a logistic regression model
model = LogisticRegression()
model.fit(features_train, target_train)

# Make a prediction for the next day
next_day_features = features.iloc[-1].values.reshape(1, -1)
next_day_features = scaler.transform(next_day_features)
next_day_prediction = model.predict(next_day_features)

# Get the Open price for the next day
next_day_open_price = df['Open'].iloc[-1]

if next_day_prediction[0] == 1:
    print(f"The model predicts a price increase. Consider buying at the open price of {next_day_open_price}.")
else:
    print(f"The model predicts a price decrease. Consider shorting at the open price of {next_day_open_price}.")


The model predicts a price increase. Consider buying at the open price of 1307.6.


c:\Users\PC\anaconda3\envs\pyenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [87]:
# Based on History Price, Calculate the percentage that next_day_prediction is correct
# Make predictions on the test set
predictions = model.predict(features_test)

# Calculate the accuracy
accuracy = (predictions == target_test).mean()

# Print the accuracy as a percentage
print(f"The model's accuracy on the test set is {accuracy * 100:.2f}%")

The model's accuracy on the test set is 57.34%


In [90]:
# Filter the DataFrame for the year 2024
df_2024 = df[df['Date'].dt.year == 2024]

# Initialize cumulative points
cumulative_points = 0

# Iterate over each day in 2024
for i in range(len(df_2024) - 1):
    # Get the features for the current day
    current_day_features = df_2024[['Open', 'High', 'Low', 'Close']].iloc[i].values.reshape(1, -1)
    current_day_features_df = pd.DataFrame(current_day_features, columns=['Open', 'High', 'Low', 'Close'])
    current_day_features_scaled = scaler.transform(current_day_features_df)
    
    # Make a prediction for the current day
    prediction = model.predict(current_day_features_scaled)[0]
    
    # Get the actual open and close prices for the next day
    next_day_open = df_2024['Open'].iloc[i + 1]
    next_day_close = df_2024['Close'].iloc[i + 1]
    
    # Calculate points based on the prediction
    if prediction == 1:  # Predicted price increase
        points = next_day_close - next_day_open
    else:  # Predicted price decrease
        points = next_day_open - next_day_close
    
    # Add the points to the cumulative points
    cumulative_points += points

print(f"Cumulative points for the year 2024: {cumulative_points}")

Cumulative points for the year 2024: 156.00000000000045
